# Topic 1.1: Geometrical transformations (기하학적 변환)

This notebook combines theory with exercises to support the understanding of geometrical transformations in medical image analysis. 

이 노트북은 이론과 연습을 결합하여 의료 이미지 분석의 기하학적 변환에 대한 이해를 지원합니다.

Implement all functions in the `code` folder of your cloned repository, and test it in this notebook after implementation by importing your functions to this notebook. Use available markdown sections to fill in your answers to questions as you proceed through the notebook.

**Contents:** <br>

1. [Review of linear algebra](#linalg)<br>
2. [Introduction to medical image registration](#intromir)<br>
   2.1 [Applications of registration](#applications)<br>
   2.2 [Classification of registration methods](#classification)<br>
   2.3 [Causes of medical image misalignment](#misalignment)<br>
3. [Geometrical transformations (theory and exercises)](#geomtrans)<br>
    3.1 [Rigid transformations](#rigtrans)<br>
    3.2 [Nonrigid transformations](#nonrigtrans)<br>
    3.3 [Transform composition](#transcomp)<br>
    3.4 [Homogeneous coordinates](#coordinates)<br>
    
**References:** <br>

[1] Fitzpatrick, J.M., Hill, D.L. and Maurer Jr, C.R., Image registration. [LINK](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)
   - Rigid transformations: [Fitzpatrick, J.M., et al. Image registration, section 8.2.1](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)
   - Non-rigid transformations: [Fitzpatrick, J.M., et al. Image registration, section 8.2.2](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)
   
[2] Kolter, Z. Do, C. Linear algebra review and reference. [LINK](http://cs229.stanford.edu/section/cs229-linalg.pdf)

[3] Maintz JB, Viergever MA. A survey of medical image registration. Med Image Anal. 1998;2(1):1–36. [LINK](https://pubmed.ncbi.nlm.nih.gov/10638851/)

In [ ]:
%load_ext autoreload
%autoreload 2

<div id='linalg'></div>

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

## 1. Review of linear algebra (선형대수학)

For animated explanation of linear algebra, you may refer to many YouTube channels, e.g. [Essence of linear algebra](https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab). It is recommended that you also read [chapters 1-3 of the document by Kolter, Z. Do, C. Linear algebra review and reference](http://cs229.stanford.edu/section/cs229-linalg.pdf).


### Scalars (스칼라)

A scalar is a single number, and can be represented by integers, real numbers, rational numbers, etc. Scalars are denoted with italic font: *a*, *n*, *t*. 

스칼라는 단일 숫자이며 정수, 실수, 유리수 등으로 표시될 수 있습니다. 스칼라는 기울임꼴 글꼴(a, n, t)로 표시됩니다.

### Vectors (벡터)

In mathematical terms, a vector is a 1-D array of numbers, while in physics terms, a vector is simply an arrow pointing in space, defined by its length and direction. While in physics, vectors do not have a common origin, in linear algebra, they typically start from the root of a coordinate system (e.g. x,y,z). Computer scientists define vectors as ordered lists of numbers. Vectors can be real, binary, integer, etc. An example notation for type and size of real vectors: $\mathbb{R}^n$

수학적인 용어로 벡터는 숫자의 1차원 배열인 반면, 물리학 용어로 벡터는 단순히 길이와 방향으로 정의되는 공간을 가리키는 화살표입니다.

<font size="3">
$$
\begin{align}
x &= \begin{bmatrix}
x_{1} \\
x_{2} \\
\vdots \\
x_{n}
\end{bmatrix}
\end{align}
$$
</font>

In Python, we can create a vector of three elements using numpy (np) arrays or the function arange. 

In [ ]:
import numpy as np # importing the numpy library

x = np.array([0,1,2]) # arrays are type-set in square brackets
print(f'\nVector x is:\n {x}')

y = x.reshape(3,1) # the reshape function takes rows and columns as input arguments
print(f'\nVector y is:\n {y}. It has length of {len(y)} elements, and shape {y.shape}.')


### Matrices (행렬)

A matrix is a finite-dimensional rectangular array of numbers arranged in rows and columns. Whenever referring to a matrix element, it is common to first list row ($i$) before collumn ($j$) indices. 

행렬은 행과 열로 배열된 숫자의 유한차원 직사각형 배열입니다. 행렬 요소를 참조할 때마다 열($j$) 인덱스보다 첫 번째 행($i$)을 나열하는 것이 일반적입니다.

The type and shape of matrix $A$ can be denoted as <font size="3">$\pmb{A} \in \mathbb{R}^{m\,\times\,n}$</font>, where elements are arranged as follows:

<br>
<font size="3">
\begin{align}
\pmb{A} &= \begin{bmatrix}
A_{1,1}\,\,\,\,A_{1,2} \\
A_{2,1}\,\,\,\,A_{2,2}
\end{bmatrix}
\end{align}
</font>

In Python, we can create a matrix using the np.matrix() function, specifying row elements in comma-separated blocks:

In [ ]:
M = np.matrix([[1,2],[3,4]])
print("\nMatrix M: \n", M)

### *Matrix (Dot) Product* (*행렬(도트) 제품*)

There are various operations that are done on matrices. Besides addition and subtraction, these operations can include multiplication by a scalar, a vector or another matrix. Scalar multiplication means that each element of a matrix is multiplied by a scalar. 

행렬에서 수행되는 다양한 작업이 있습니다. 덧셈과 뺄셈 외에도 이러한 연산에는 스칼라, 벡터 또는 다른 행렬에 의한 곱셈이 포함될 수 있습니다. 스칼라 곱셈은 행렬의 각 요소에 스칼라를 곱하는 것을 의미합니다.

If we multiply an $m \times n$ matrix by a vector, then the output is a linear combination of all columns ($\pmb{C} = \pmb{AB}$, where $\pmb{C}_{i,j} = \sum_{k}\,\pmb{A}_{i,k}\,\pmb{B}_{k,j}$). It is however important to realize that _order matters_, i.e. matrix multiplication is not commutative ($\pmb{AB} \neq \pmb{BA}$). 

In [ ]:
# Example of matrix multiplication

A = np.matrix([[1,2,1], [0,2,1]])
B = np.matrix([[1,2,0], [0,3,1], [-2,1,1]])

# Possible multiplication
print("\nMultiplication of A*B: \n", A*B)

# Multiplication is not commutative (you can check the output error by uncommenting the line below)
#print("\nMultiplication of B*A: \n", B*A)


### *Matrix transpose* (*행렬 전치*)

The transpose of a matrix can be thought of as a mirror image across the main diagonal. The first column becomes the first row, the second column becomes the second row, etc. 

행렬의 전치는 주대각선을 가로지르는 거울 이미지로 생각할 수 있습니다. 첫 번째 열은 첫 번째 행이 되고, 두 번째 열은 두 번째 행이 됩니다.

An $n \times m$ matrix is said to be symmetric if $\mathbf{A} = \mathbf{A}^{\top}$, and skew symmetric if $\mathbf{A}^{\top} = - \mathbf{A}$. In other words, <font size="3">$(\pmb{A}^\top)_{i,j} = \pmb{A}_{i,j}$</font>, and <font size="3">$(\pmb{AB})^\top = \pmb{B}^\top\pmb{A}^\top$</font>.

In [ ]:
# Matrix transpose is done using the transpose() function 
M = np.matrix([[1,2],
               [3,4]])

print("\nMatrix M: \n", M)
print("\nMatrix M transposed: \n", M.transpose())

### *Identity matrix* (*단위 행렬*)

An identity matrix of size $n \times n$ is a square matrix with ones on its main diagonal and all other elements equal to zero,

$n \times n$ 크기의 단위 행렬은 주대각선에 1이 있고 다른 모든 요소는 0인 정사각 행렬입니다.

i.e.
<font size="3">
$\forall\pmb{x}\in \mathbb{R}^{n},\pmb{I}_{n}\pmb{x} = \pmb{x}$.<br>
    
$$
\begin{align}
\pmb{I} &= \begin{bmatrix}
1 \,\,\,\, 0 \,\,\,\, 0 \\
0 \,\,\,\, 1 \,\,\,\, 0 \\
0 \,\,\,\, 0 \,\,\,\, 1 
\end{bmatrix}
\end{align}
$$
</font>

In [ ]:
# Identity matrix can be created using the eye() or identity () function
N = np.eye(3)
M = np.identity(2, dtype = float) #dtype determines the data type of a variable

print("\nMatrix N with eye(): \n", N)
print("\nMatrix M with identity(): \n", M)

### *Matrix inversion* (*행렬 반전*)

A matrix inverse is a matrix which when multiplied by the original matrix will yield an identity matrix. 

역행렬은 원래 행렬을 곱하면 단위 행렬이 되는 행렬입니다.

Matrix inverse is denoted as: <font size="3">$\pmb{A}^{-1}\pmb{A} = \pmb{I}_{n}$</font>. However, not all matrices have their inverse form. The inverse of a matrix exists only if the matrix is non-singular ($\det A \neq 0$). A system of equations can be solved using matrix inverse as follows:

<font size="3">
$$
\begin{matrix}
\begin{align}
\pmb{Ax} &= b \\
\pmb{A}^{-1}\pmb{Ax} &= \pmb{A}^{-1}\pmb{b} \\
\pmb{I}_{n}\pmb{x} &= \pmb{A}^{-1}\pmb{b} \\
\end{align}
\end{matrix}
$$
</font>


In [ ]:
# Matrix inversion can be calculated using the numpy function linalg.inv():

# Taking a 3 * 3 matrix
A = np.array([[6, 1, 1], [4, -2, 5], [2, 8, 7]])

A_inv =  np.linalg.inv(A)
# Calculating the inverse of the matrix
print("\nThe inverse of matrix A is: \n", A_inv.round(3))

### Special matrices and vectors (특수 행렬 및 벡터)

There is a plethora of special matrices and vectors in linear algebra, the explanation of which is beyond the scope of this notebook. 

선형대수학에는 특별한 행렬과 벡터가 너무 많아서 이에 대한 설명은 이 노트의 범위를 벗어납니다.

To name a couple of examples, a unit vector is a vector of length 1: <font size="3">$||\pmb{x}||_{2} = 1$</font>; a symmetric matrix is a matrix which is equal to its transposed form: <font size="3">$\pmb{A} = \pmb{A}^\top$</font>; and a square matrix is said to be orthogonal or orthonormal if its transpose is equal to the inverse (<font size="3">$\pmb{A}^{-1} = \pmb{A}^\top$</font>) of that matrix: <font size="3">$\pmb{A}^\top\pmb{A} = \pmb{A}\pmb{A}^\top = \pmb{I}$</font>. 
    

### Systems of equations (방정식 시스템)

Via the so-called augmented matrices, one can solve systems of equations. 

소위 증강 행렬을 통해 방정식 시스템을 풀 수 있습니다.

This process is not vastly different from what you normally do when solving single equations. Augmented matrices contain all equation arguments as rows. The operations on these rows consist of switching two rows, multiplication of a row by a nonzero number, and replacing a row by a multiple of another row added to it. Any row operation can be undone by another inverse row operation. Here, we show an example in Python to solve the following system of equations:

<br>
<font size="3">
$$
\begin{matrix}
\begin{align}
8x + 3y - 2z &= 9 \\
-4x + 7y + 5z &= 15 \\
3x + 4y - 12z &= 35
\end{align}
\end{matrix}
$$
</font>

In [ ]:
# Solving systems of equations is easy using the numpy function linalg.solve():

A = np.array([[8, 3, -2], [-4, 7, 5], [3, 4, -12]])
b = np.array([9, 15, 35])
x = np.linalg.solve(A, b)

print("\nSolution for given system of equations [x,y,z] is: \n", x.round(2))

### Norms (규범)

Norms are defined as functions that measure the magnitude of a matrix or vector. 

노름은 행렬이나 벡터의 크기를 측정하는 함수로 정의됩니다.

E.g. the distance of a vector from its origin is called a Euclidean norm, which can also be defined as the square root of the inner product of a vector with itself. The norm of a matrix expresses the magnitude of that matrix regardless of the number of its elements. Vector norms have the following three properties: 

<br>
<font size="4">
$$
\begin{matrix}
\begin{align}
f(\pmb{x}) &= 0 \Longrightarrow \pmb{x} = 0 \\
f(\pmb{x + y}) &\leq f(\pmb{x}) + f(\pmb{y})\,\,\,\,\text{(triangle inequality)} \\
\forall\alpha \in \mathbb{R}, f(\alpha\pmb{x}) &= |\alpha|f(\pmb{x}) \\
\end{align}
\end{matrix}
$$
</font>


In [ ]:
# Norms can be calculated in Python using the numpy function linalg.norm()
# This function returns one of the seven matrix norms 
# or one of the infinite vector norms depending upon the value of its parameters.

# initialize vector
x = np.arange(20)
 
# compute norm of vector
x_norm = np.linalg.norm(x)
 
print("\nVector norm of x is: \n", x_norm.round(2))

### Determinant (결정자)

Determinant of a matrix is a special number defined only for square matrices, representing the matrix in terms of a real number which can be used to solve systems of linear equations and finding matrix inverse. 

행렬식은 정사각형 행렬에 대해서만 정의된 특수 숫자로, 선형 방정식 시스템을 풀고 역행렬을 찾는 데 사용할 수 있는 실수로 행렬을 나타냅니다.

Determinant of a transformation matrix $\textbf{T}$ is the signed area of a unit square shape after transforming with $\textbf{T}$. The sign reflects whether the orientation has changed or not. The determinant of a $2 \times 2$ matrix is calculated as the subtraction of cross-diagonal element multiplication. It is common to use the absolute value of the determinant: 

$$
\begin{align}
\det\begin{pmatrix} a & b \\ c & d \end{pmatrix} = |ad - bc|
\end{align}
$$

In [ ]:
# The determinant of a matrix can be calculated using the numpy function linalg.det():

M = np.matrix([[1,2],[3,4]])

M_det = abs(np.linalg.det(M))

print("\nDeterminant of matrix M: \n", M_det)


---

<div id='intromir'></div>
<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

##  2. Introduction to (medical) image registration ((의료)영상등록 소개)

Image registration is the determination of a geometrical transformation that aligns one view of an object with another view of that object or another object. The term "view" refers to a two-, three-dimensional image or the physical representation of an object in space. An example of two-dimensional image types may be x-ray projections captured as a digital radiograph or a light projection in a video frame. Three-dimensional images can be collected by imaging modalities commonly used in hospital settings, e.g. computed tomography (CT) or magnetic resonance (MR) imaging scanners. Generally, images are stored as discrete arrays of intensity values, and in medical applications, the object in each view will represent an anatomical region of interest. Explained mathematically, the inputs of registration are two views, which we map together by matching points positioned in one view to points in another view. 

이미지 등록은 객체의 한 뷰를 해당 객체의 다른 뷰 또는 다른 객체와 정렬하는 기하학적 변환을 결정하는 것입니다. "뷰"라는 용어는 2차원, 3차원 이미지 또는 공간에 있는 물체의 물리적 표현을 의미합니다. 2차원 이미지 유형의 예로는 디지털 방사선 사진으로 캡처된 X선 투영이나 비디오 프레임의 광 투영이 있을 수 있습니다. 병원 환경에서 일반적으로 사용되는 영상 기법을 통해 3차원 영상을 수집할 수 있습니다. 컴퓨터 단층촬영(CT) 또는 자기공명(MR) 영상 스캐너. 일반적으로 이미지는 강도 값의 개별 배열로 저장되며 의료 응용 분야에서는 각 보기의 개체가 해부학적 관심 영역을 나타냅니다. 수학적으로 설명하면 등록 입력은 두 개의 보기이며, 한 보기에 위치한 점을 다른 보기의 점과 일치시켜 함께 매핑합니다.

<div id='applications'></div>

### Applications of registration (이미지 등록신청)

Registration may be applied to various purposes. It allows us to combine information from different sources (MR-guided radiotherapy planning) or investigate longitudinal changes in e.g. post-treatment patient monitoring. Moreover, registration procedures are employed when studying group changes across multiple subjects in a trial. Last but not least, registration can aid in segmentation tasks when mapping atlases with anatomical model priors to a newly acquired medical image, or when performing e.g. motion-induced image artefact corrections.

등록은 다양한 목적으로 적용될 수 있습니다. 이를 통해 다양한 소스(MR 유도 방사선 치료 계획)의 정보를 결합하거나 종단적 변화를 조사할 수 있습니다. 치료 후 환자 모니터링. 더욱이, 시험에서 여러 주제에 걸쳐 그룹 변경을 연구할 때 등록 절차가 사용됩니다. 마지막으로 등록은 새로 획득한 의료 이미지 이전에 해부 모델로 아틀라스를 매핑하거나 다음과 같은 작업을 수행할 때 분할 작업에 도움이 될 수 있습니다. 모션으로 인한 이미지 아티팩트 수정.

<div id='classification'></div>

### Classification of registration methods (등록방법 분류)

There is a complex categorization of registration methods in the field. The following eight categories have been proposed in relevant literature ([Fitzpatrick, J.M., et al. Image registration, section 8.1.2](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)): image dimensionality, registration basis, geometrical transformation, degree of interaction, optimization procedure, modalities, subject, and object. In medical applications, we typically work with individual two-dimensional slices or three-dimensional image volumes, which may be acquired sequentially over time or as a series of multiple 3D volumes (e.g. diffusion MR images). Registration may be performed via various bases, using either a location in respective views (point-based registration) or intensity similarities (intensity-based registration). Further classification of registration methods can be based on geometrical transformations, i.e. which geometrical manipulation (rigid, affine, nonlinear, etc.) are applied for alignment between two different spaces. Registration can be either automatic or semi-automatic, depending on the amount of human interaction during the registration process. The quality of registration output is estimated continuously during the procedure either as a closed-form solution or iteratively. At last, registration methods are stratified according to the amount of modalities they involve (multi-modal, intra-modal), the subjects involved in a trial (inter-subject, intra-patient, atlas-based), and commonly also the anatomical object of interest (e.g. brain, liver, etc.).

해당 분야에는 등록 방법이 복잡하게 분류되어 있습니다. 다음 8가지 범주가 관련 문헌에서 제안되었습니다([Fitzpatrick, J.M., et al. Image Registration, 섹션 8.1.2](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464. 5408&rep=rep1&type=pdf)): 이미지 차원, 등록 기준, 기하학적 변환, 상호 작용 정도, 최적화 절차, 양식, 주제 및 객체. 의료 응용 분야에서 우리는 일반적으로 개별 2차원 슬라이스 또는 3차원 이미지 볼륨을 사용하여 작업하며, 이는 시간이 지남에 따라 순차적으로 또는 일련의 여러 3D 볼륨(예: 확산 MR 이미지)으로 획득될 수 있습니다. 등록은 각 뷰의 위치(포인트 기반 등록) 또는 강도 유사성(강도 기반 등록)을 사용하여 다양한 기반을 통해 수행될 수 있습니다. 등록 방법의 추가 분류는 기하학적 변환, 즉 기하학적 조작(강성, 아핀, 비선형 등)이 두 개의 서로 다른 공간 간의 정렬에 적용되는 것을 기반으로 할 수 있습니다. 등록은 등록 프로세스 중 인간 상호 작용의 양에 따라 자동 또는 반자동으로 이루어질 수 있습니다. 등록 출력의 품질은 절차 중에 폐쇄형 솔루션이나 반복적으로 지속적으로 추정됩니다. 마지막으로, 등록 방법은 관련된 양식(다중 모드, 내부 모드), 시험에 관련된 피험자(피험자 간, 환자 내, 아틀라스 기반) 및 일반적으로 해부학적 방식의 양에 따라 계층화됩니다. 관심 대상(예: 뇌, 간 등).

<div id='misalignment'></div>

### Causes of medical image misalignment (의료영상 정렬 불량의 원인)

There are various reasons for misalignments across multiple images / volumes in a series or two images acquired at different time points. In measurements where patient compliance is crucial, different patient positioning, physiological movements of organs (heartbeat, breathing, cerebrospinal fluid flow), patient motion during image acquisition, and distortions caused by imaging systems (e.g. due to the design of imaging sequences in magnetic resonance imaging) can cause data misalignment. In cases where image acquisition includes interventions (e.g. surgery, chemotherapy, biopsy), users benefit from image registration as well.

시리즈의 여러 이미지/볼륨 또는 서로 다른 시점에서 획득한 두 개의 이미지 간에 정렬이 잘못되는 데는 다양한 이유가 있습니다. 환자 순응도가 중요한 측정에서는 다양한 환자 위치, 장기의 생리적 움직임(심장 박동, 호흡, 뇌척수액 흐름), 영상 획득 중 환자 움직임, 영상 시스템으로 인한 왜곡(예: 자기 공명 영상 시퀀스 설계로 인해) 이미징)은 데이터 정렬 오류를 일으킬 수 있습니다. 이미지 획득에 개입(예: 수술, 화학 요법, 생검)이 포함되는 경우 사용자는 이미지 등록의 이점도 누릴 수 있습니다.

An example of how digital subtraction angiography benefits from image registration:

디지털 차감 혈관조영술이 이미지 등록을 통해 어떤 이점을 얻을 수 있는지에 대한 예:

<center width="100%"><img src="../labs/assets/digital_subtraction_angiography.png" width="700"></center>

---

<div id='geomtrans'></div>
<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

## 3. Geometrical transformations (theory and exercises) (기하학적 변환(이론 및 연습))

In registration operations, each involved view is defined by a coordinate system. As written above, registration is the act of mapping points from space $X$ to space $Y$. A successful registration is achieved when the point $y$ in space $Y$ is approximately equal or completely correspondent to $x^{\prime}$ after a transformation $T$ has been applied to point $x$ in $X$. In the picture below, geometrical transformation $T$ aligns the moving axial brain scan with the fixed one.

등록 작업에서 관련된 각 뷰는 좌표계에 의해 정의됩니다. 위에서 설명한 것처럼 등록은 $X$ 공간에서 $Y$ 공간으로 포인트를 매핑하는 행위입니다. 변환 $T$가 $X$의 $x$ 지점에 적용된 후 $Y$ 공간의 $y$ 지점이 $x^{\prime}$과 거의 같거나 완전히 일치하면 성공적인 등록이 이루어집니다. 아래 그림에서 기하학적 변환 $T$는 움직이는 축 뇌 스캔을 고정된 스캔과 정렬합니다.

<center width="100%"><img src="../labs/assets/fixed_moving.png" width="700"></center>

<div id='rigtrans'></div>

## 3.1 Rigid transformations (강체 변환)

Rigid transformations ([Fitzpatrick, J.M., et al. Image registration, section 8.2.1](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)) are geometrical alignments of two objects that preserve distances, the planarity of surfaces and angles between straight lines. The so-called rigid registration problems involve object translation and rotation. 

강체 변환([Fitzpatrick, J.M., et al. Image Registration, 섹션 8.2.1](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf))은 다음과 같습니다. 거리, 표면의 평면성 및 직선 사이의 각도를 보존하는 두 개체의 기하학적 정렬입니다. 소위 엄격한 등록 문제에는 객체 이동 및 회전이 포함됩니다.

### Translation (이동)

Translation is arguably the "simplest" geometrical transformation that can be applied to an object. 

평행 이동은 객체에 적용할 수 있는 "가장 간단한" 기하학적 변환이라고 할 수 있습니다.

Assuming that the coordinates of a 2D geometric object are stored in the variable $X$ (the first row contains the horizontal coordinates and the second row contains the vertical coordinates), translation of the geometric object can be performed by adding a 2D translation vector $X_{t}$ to every vertex of $X$, as shown in the Python example below.

In [ ]:
import numpy as np

# An example of translation in Python:
X = np.matrix([[1,2,3],[1,4,6]])
Xt = [4,5]

X[0,:] = X[0,:] + Xt[0]
X[1,:] = X[1,:] + Xt[1]

print("\nTranslation of the first vector X[0,:]: \n", X[0,:])
print("\nTranslation of the second vector X[1,:]: \n", X[1,:])

### Rotation (회전)

Image rotation is a rigid transformation that requires a rotation angle $\theta$ defining the number of degrees for rotation. Typically, rotation is done about image origin (e.g. $x_{0}, y_{0}$).

이미지 회전은 회전 각도를 정의하는 회전 각도 $\theta$가 필요한 엄격한 변환입니다. 일반적으로 회전은 이미지 원점(예: $x_{0}, y_{0}$)을 기준으로 수행됩니다.

In [ ]:
# An example of rotation in Python using numpy and scipy:
X = np.matrix([[1,2,3],[1,4,6]])

X_rotated_90 = np.rot90(X)

from scipy.ndimage import rotate
X_rotated_270 = rotate(X, angle = 270, reshape=True)

print("\nRotation by 90 degrees: \n", X_rotated_90)
print("\nRotation by 270 degrees: \n", X_rotated_270)

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

<div id='nonrigtrans'></div>
 
## 3.2 Nonrigid transformations (비강체 변환)

Nonrigid transformations ([Fitzpatrick, J.M., et al. Image registration, section 8.2.2](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf)) are essential in registration operations for interpatient comparisons of rigid anatomies, as well as intrapatient registration of anatomical structures with artifactual distortions induced during image acquisition of nonrigid anatomies (e.g. MRI scans of beating heart). Nonrigid transformations include scaling, where the straightness of lines and the angles between them are preserved (used e.g. to suppress calibration errors in MR scanners), and affine transformations, where the angle between lines may be changed (used e.g. in deskewing a CT image after improper gantry angle recording). Further examples of nonrigid registrations comprise projective, perspective and curved registration methods (see chapters...)

비강성 변환([Fitzpatrick, J.M., et al. Image Registration, 섹션 8.2.2](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.464.5408&rep=rep1&type=pdf))은 다음과 같습니다. 강성 해부학의 환자 간 비교를 위한 등록 작업과 비강성 해부학의 이미지 획득 중에 유발된 인공 왜곡이 있는 해부학 구조의 환자 내 등록(예: 심장 박동의 MRI 스캔)에 필수적입니다. 비강성 변환에는 선의 직선성과 선 사이의 각도가 유지되는 스케일링(예: MR 스캐너의 교정 오류를 억제하는 데 사용됨)과 선 사이의 각도가 변경될 수 있는 아핀 변환(예: 후 CT 이미지 기울기를 조정하는 데 사용됨)이 포함됩니다. 부적절한 갠트리 각도 기록). 비강성 등록의 추가 예에는 투영, 원근 및 곡선 등록 방법이 포함됩니다(장 참조).


Let us leave translation aside for now and focus on the other, more complex geometrical transformations. The identity, scaling, reflection and shearing transformations (or any combination of these transformations) can be performed by multiplying the matrix of coordinates $X$ with an appropriate transformation matrix $T$. Here is an example of Python code that compute transformation matrices for the identity transformation (which is not really a transformation) and scaling:

지금은 번역을 제쳐두고 다른 좀 더 복잡한 기하학적 변환에 초점을 맞춰 보겠습니다. 항등, 스케일링, 반사 및 전단 변환(또는 이러한 변환의 조합)은 좌표 행렬 $X$에 적절한 변환 행렬 $T$를 곱하여 수행할 수 있습니다. 다음은 항등 변환(실제로는 변환이 아님) 및 스케일링에 대한 변환 행렬을 계산하는 Python 코드의 예입니다.

### Scaling (스케일링 (크기 변화기))

Scaling can be performed for example in the following way: `X_scaled = scale(2,3)*X`. 

스케일링은 예를 들어 `X_scaled = scale(2,3)*X`와 같은 방식으로 수행될 수 있습니다.

To verify this, we can use the provided `test_object()` function in the `registration_util.py` module that returns a test geometrical object in the shape of the letter *F*, and plot the original object and a scaled version of it (the provided `plot_object()` function in the `registration_util.py` module can be used to plot the geometrical object) as follows:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import registration as reg
import registration_util as util

def identity():
    T = np.eye(2)
    return T

def scale(sx, sy):
    T = np.array([[sx,0],[0,sy]])
    return T

X = util.test_object(1)
X_scaled = reg.scale(2, 3).dot(X)

fig = plt.figure(figsize=(5,5))
ax1  = fig.add_subplot(111)
ax1.grid()
util.plot_object(ax1, X)
util.plot_object(ax1, X_scaled)

### Reflection (반사)

Reflection is the mirror transformation of an image along a given axis. 

반사는 주어진 축을 따라 이미지가 거울로 변환되는 것입니다.

### Shearing (전단)

Shearing transformation (a.k.a transvection) is a type of transformation where each point is displaced by a distance proportional to the perpendicular distance from the point's parallel line. In 3D, planes are sheared instead of points.

전단 변환(일명 transvection)은 각 점이 점의 평행선으로부터 수직 거리에 비례하는 거리만큼 변위되는 변환 유형입니다. 3D에서는 점 대신 평면이 잘립니다.

In [ ]:
# An example of reflection in Python:
X = np.matrix([[1,2,3],[1,4,6]])

X_right_left = np.fliplr(X)
X_upside_down = np.flipud(X)

print(X_right_left)
print(X_upside_down)

---

## ⭐ Self Exercises - Later (개인 과제 - 나중에)

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/todo_ico.png" width="42" height="42"></div> 

### *Exercise 3.2.1*:

Implement functions that return transformation matrices for 2D rotation, shear and reflection. You can find the templates for these three function definitions in `SECTION 1` of the `registration.py` module. To test your implementation, run the `transforms_test()` script from the `registration_tests.py` module and make sure that the output matches the figure below.

2D 회전, 전단 및 반사에 대한 변환 행렬을 반환하는 함수를 구현합니다. `registration.py` 모듈의 `SECTION 1`에서 이 세 가지 함수 정의에 대한 템플릿을 찾을 수 있습니다. 구현을 테스트하려면 `registration_tests.py` 모듈에서 `transforms_test()` 스크립트를 실행하고 출력이 아래 그림과 일치하는지 확인하세요.

<center width="100%"><img src="../labs/assets/transforms_test.png" width="800"></center>

In [ ]:
%matplotlib inline
from registration_tests import transforms_test

transforms_test()

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/question_ico.png" width="42" height="42"></div> 

### *Question 3.2.1*:

What is rigid and what affine transformation? How many degrees of freedom dothese two types of transformations have in 2D?

강체변환은 무엇이고 아핀변환은 무엇입니까? 2D에서 이 두 가지 유형의 변환은 얼마나 많은 자유도를 갖습니까?

<font style="color:red">Type your answer here</font>

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/question_ico.png" width="42" height="42"></div> 

### *Question 3.2.2*:

What is the minimum number of corresponding point pairs needed to fit a 2D affine transform? How about 3D? Motivate your answer.

2D 아핀 변환을 맞추는 데 필요한 대응 점 쌍의 최소 수는 얼마입니까? 3D는 어떻습니까? 답변에 동기를 부여하십시오.

<font style="color:red">Type your answer here</font>

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

<div id='transcomp'></div>

## 3.3 Transform composition (변환 구성)

Geometrical transformations can be combined by multiplying transformation matrices. These compositions may involve rotation and translation, or even rotation, scaling, shearing, reflection and translation altogether. For example, the following (conceptual) command first applies a 90° rotation to an object and then a vertical reflection: `X_t = reg.reflect(-1,1).dot(reg.rotate(np.pi/2)).dot(X)`. 

기하학적 변환은 변환 행렬을 곱하여 결합할 수 있습니다. 이러한 구성에는 회전 및 이동이 포함될 수 있으며 심지어 회전, 크기 조정, 기울이기, 반사 및 이동이 모두 포함될 수도 있습니다. 예를 들어, 다음(개념적) 명령은 먼저 객체에 90° 회전을 적용한 다음 수직 반사를 적용합니다. `X_t = reg.reflect(-1,1).dot(reg.rotate(np.pi/2)) .점(X)`.

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/question_ico.png" width="42" height="42"></div> 

### *Question 3.3.1:*

Would the result be different if the two transformations in the example above are applied in reversed order? Motivate your answer.

위 예의 두 변환을 역순으로 적용하면 결과가 달라 집니까? 답변에 동기를 부여하십시오.


<font style="color:red">Type your answer here</font>

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/question_ico.png" width="42" height="42"></div> 

### *Question 3.3.2:*

How can you compute the inverse of an affine transformation represented with a transformation matrix?

변환 행렬로 표현된 아핀 변환의 역함수를 어떻게 계산할 수 있나요?

<font style="color:red">Type your answer here</font>

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/todo_ico.png" width="42" height="42"></div> 

### *Exercise 3.3.1*:

Test a few more examples of combining transformations. Save the examples in the `combining_transforms()` script template in the `registration_tests.py` module.

변환 결합에 대한 몇 가지 예를 더 테스트해 보세요. `registration_tests.py` 모듈의 `combining_transforms()` 스크립트 템플릿에 예제를 저장합니다.

In [ ]:
%matplotlib inline
from registration_tests import combining_transforms
combining_transforms()

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/read_ico.png" width="42" height="42"></div> 

<div id='coordinates'></div>

## 3.4 Homogeneous coordinates (동차 좌표)

As mentioned in the beginning of the previous exercise, translation can be performed by adding a translation vector to the coordinates of an object. Translation can be combined with other geometrical transformations, for example:

이전 연습의 시작 부분에서 언급한 것처럼 객체의 좌표에 이동 벡터를 추가하여 이동을 수행할 수 있습니다. 평행 이동은 다른 기하학적 변환과 결합될 수 있습니다. 예를 들면 다음과 같습니다.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import registration as reg
import registration_util as util

X = util.test_object(1)

# translation vector
t = np.array([10, 20])

# rotate by 45 deg.
X_rot = reg.rotate(np.pi/4).dot(X)

# translate by 10 in the horizontal and 20 in the vertical direction
X_rot_tran = np.empty(shape=X.shape)
X_rot_tran[0,:] = X_rot[0,:] + t[0];
X_rot_tran[1,:] = X_rot[1,:] + t[1];

fig = plt.figure(figsize=(5,5))
ax1  = fig.add_subplot(111)
ax1.grid()
util.plot_object(ax1, X)
util.plot_object(ax1, X_rot_tran)

However, this way of combining translation with other transformations can be a bit cumbersome (it somewhat complicates the mathematical notation and implementation in code). The transformations that you have implemented in the previous exercise can be straightforwardly combined with translation by converting the transformation matrix to homogeneous form. This matrix can then be applied to the homogeneous coordinates (details can be found in the lecture slides). The function `c2h()` given below (also available in the `registration_util.py` module) implements conversion from Cartesian coordinates to homogeneous coordinates. As you can see from the code, this conversion is performed by simply adding an additional row of coordinates with all ones:

그러나 번역을 다른 변환과 결합하는 이 방법은 약간 번거로울 수 있습니다(수학적 표기법과 코드 구현이 다소 복잡해집니다). 이전 연습에서 구현한 변환은 변환 행렬을 동종 형식으로 변환하여 변환과 직접 결합할 수 있습니다. 그런 다음 이 행렬을 동차 좌표에 적용할 수 있습니다(자세한 내용은 강의 슬라이드에서 확인할 수 있습니다). 아래 제공된 `c2h()` 함수(`registration_util.py` 모듈에서도 사용 가능)는 데카르트 좌표에서 동차 좌표로의 변환을 구현합니다. 코드에서 볼 수 있듯이 이 변환은 모든 좌표 행에 추가 좌표 행을 추가하여 수행됩니다.

In [ ]:
%matplotlib inline
import registration_util as util

X = util.test_object(1)
Xh = util.c2h(X)

print('X:\n{}\n'.format(X))
print('Xh:\n{}\n'.format(Xh))

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/todo_ico.png" width="42" height="42"></div> 

### *Exercise 3.4.1*:

Implement the function called `t2h()` in the `registration_util.py` module that converts a transformation matrix and a translation vector to a transformation matrix in homogeneous form. The template for this definition is already provided in the module file. To test your function, verify that the `t2h_test()` script results in the same object as the example above (note that the function `plot_object()` also works with homogeneous coordinates):

변환 행렬과 변환 벡터를 동차 형태의 변환 행렬로 변환하는 `t2h()`라는 함수를 `registration_util.py` 모듈에 구현합니다. 이 정의에 대한 템플릿은 이미 모듈 파일에 제공되어 있습니다. 함수를 테스트하려면 `t2h_test()` 스크립트가 위의 예와 동일한 객체를 생성하는지 확인하세요(`plot_object()` 함수도 동종 좌표에서도 작동한다는 점에 유의하세요).

In [ ]:
%matplotlib inline
from registration_tests import t2h_test

t2h_test()

The rotation transformation rotates the objects counterclockwise around the origin of the coordinate system. To perform rotation around an arbitrary point, the following sequence of transformations must be applied:

1. Translate the object so the arbitrary rotation point is translated to the origin of the coordinate system
2. Rotate the object
3. Translate the object back so that the arbitrary rotation point is in the original location.

These three transformations can be combined by multiplying the three homogeneous transformation matrices. Combining transformation matrices in homogeneous form works in the same way as the "regular" transformation matrices, i.e. by matrix multiplication. 

회전 변환은 좌표계 원점을 중심으로 객체를 시계 반대 방향으로 회전합니다. 임의의 점을 중심으로 회전을 수행하려면 다음과 같은 변환 순서를 적용해야 합니다.

1. 임의의 회전점이 좌표계의 원점으로 변환되도록 객체를 변환합니다.
2. 개체 회전
3. 임의의 회전점이 원래 위치에 있도록 개체를 다시 이동합니다.

이 세 가지 변환은 세 개의 동종 변환 행렬을 곱하여 결합할 수 있습니다. 동종 형태의 변환 행렬 결합은 "일반" 변환 행렬과 동일한 방식, 즉 행렬 곱셈을 통해 작동합니다.

# <div style="float:right;margin:-5px 5px"><img src="../labs/assets/todo_ico.png" width="42" height="42"></div> 

### *Exercise 3.4.2*:

Write an example that rotates the test object by 45° around the first vertex (hint: the first vertex is `X[:,0]` and `t2h(reg.identity(), Xt)` is a homogeneous transformation matrix that performs only translation). Save the example in the provided `arbitrary_rotation()` template in the `registration_tests.py` module. The result should match the one shown in the figure below.

첫 번째 꼭지점을 중심으로 테스트 개체를 45° 회전시키는 예제를 작성하세요(힌트: 첫 번째 꼭지점은 `X[:,0]`이고 `t2h(reg.identity(), Xt)`는 다음을 수행하는 동종 변환 행렬입니다. 번역만 가능). `registration_tests.py` 모듈에 제공된 `arbitrary_rotation()` 템플릿에 예제를 저장합니다. 결과는 아래 그림에 표시된 것과 일치해야 합니다.

<center width="100%"><img src="../labs/assets/arbitrary_rotation.png" width="400"></center>


In [ ]:
%matplotlib inline

import sys
sys.path.append("../code")
from registration_tests import arbitrary_rotation

arbitrary_rotation()

<div style="float:right;margin:-5px 5px"><img src="../labs/assets/question_ico.png" width="42" height="42"></div> 

### *Question 3.4.1:*

Assuming you have implemented the missing functionality correctly, will the following line of code result in an apparent clockwise or counter-clockwise rotation of the image? Motivate your answer. (Hint: think about the coordinate system of the image, also shown in the figures illustrating forward and inverse mapping above.)

누락된 기능을 올바르게 구현했다고 가정하면 다음 코드 줄은 이미지가 시계 방향 또는 시계 반대 방향으로 회전하게 됩니까? 답변에 동기를 부여하십시오. (힌트: 위의 순방향 매핑과 역방향 매핑을 설명하는 그림에도 표시된 이미지의 좌표계를 생각해 보세요.)

```python
It = image_transform(I, t2h(rotate(pi/4), [0 0]))
``` 

<font style="color:red">Type your answer here</font>